In [13]:
from sympy import *
init_printing(use_unicode=True)

In [14]:
# STATE
x = Symbol('x')
y = Symbol('y')
theta = Symbol('theta')
l_ratio = Symbol('l_ratio')
d_ratio = Symbol('d_ratio')
h = Symbol('h')
v = Symbol('v')
w = Symbol('w')
state = Matrix([x,y,theta,l_ratio,d_ratio,h,v,w])

l = l_ratio*h
d = d_ratio*h

# VIEW MATRIX
# vr00 = Symbol('Vr_00')
# vr01 = Symbol('Vr_01')
# vr02 = Symbol('Vr_02')
# vr10 = Symbol('Vr_10')
# vr11 = Symbol('Vr_11')
# vr12 = Symbol('Vr_12')
# vr20 = Symbol('Vr_20')
# vr21 = Symbol('Vr_21')
# vr22 = Symbol('Vr_22')
# VR = Matrix([[vr00, vr01, vr02],[vr10, vr11, vr12],[vr20, vr21, vr22]])
# vtx = Symbol('Vtx')
# vty = Symbol('Vty')
# vtz = Symbol('Vtz')
vr00 = Symbol('vr00')
vr01 = Symbol('vr01')
vr02 = Symbol('vr02')
vr10 = Symbol('vr10')
vr11 = Symbol('vr11')
vr12 = Symbol('vr12')
vr20 = Symbol('vr20')
vr21 = Symbol('vr21')
vr22 = Symbol('vr22')
VR = Matrix([[vr00, vr01, vr02],[vr10, vr11, vr12],[vr20, vr21, vr22]])
vtx = Symbol('vtx')
vty = Symbol('vty')
vtz = Symbol('vtz')
VT = Matrix([vtx,vty,vtz])
viewMat = eye(4)
viewMat[0:3,0:3] = VR
viewMat[0:3,3] = VT

# PROJECTION MATRIX
fx = Symbol('fx')
fy = Symbol('fy')
cx = Symbol('cx')
cy = Symbol('cy')
P = Matrix([[fx,0,cx,0],[0,fy,cy,0],[0,0,1,0]])

# WORLD VIEW PROJECTION MATRIX
m00 = Symbol('m_00')
m01 = Symbol('m_01')
m02 = Symbol('m_02')
m03 = Symbol('m_03')
m10 = Symbol('m_10')
m11 = Symbol('m_11')
m12 = Symbol('m_12')
m13 = Symbol('m_13')
m20 = Symbol('m_20')
m21 = Symbol('m_21')
m22 = Symbol('m_22')
m23 = Symbol('m_23')
M_wvp = Matrix([[m00, m01, m02, m03],[m10, m11, m12, m13],[m20, m21, m22, m23]])

In [15]:
# ELLIPSOID
z = Symbol('z')

a = Symbol('a_3D')
b = Symbol('b_3D')
c = Symbol('c_3D')

subs_ellipsoid = {
    z : h/2,
    a : l/2,
    b : d/2,
    c : h/2
}

Q = Matrix([[a**2, 0, 0, 0], [0, b**2, 0, 0],[0, 0, c**2, 0],[0, 0, 0, -1]])

# WORLD Matrix
WR = Matrix([[cos(theta), -sin(theta), 0],[sin(theta), cos(theta), 0],[0,0,1]])
Re_w = eye(4)
Re_w[0:3,0:3] = WR.T

center_ellipsoid = Matrix([x,y,z])
T_c_inv = eye(4)
T_c_inv[0:3,3] = center_ellipsoid

W = T_c_inv*Re_w.T

# world view projection matrix
M_wvp_real = P*viewMat*W

subs_wvp = {
    m00 : M_wvp_real[0,0],
    m01 : M_wvp_real[0,1],
    m02 : M_wvp_real[0,2],
    m03 : M_wvp_real[0,3],
    m10 : M_wvp_real[1,0],
    m11 : M_wvp_real[1,1],
    m12 : M_wvp_real[1,2],
    m13 : M_wvp_real[1,3],
    m20 : M_wvp_real[2,0],
    m21 : M_wvp_real[2,1],
    m22 : M_wvp_real[2,2],
    m23 : M_wvp_real[2,3]
}

# Compute the check to perform to see if an object is behind the camera
M_wv = viewMat*W
Q_star_camera = M_wv * Q * M_wv.T
check = Q_star_camera[2,3]/Q_star_camera[3,3]
# if check < 0 the object is behind the camera 

In [16]:
# ELLIPSE
C_star = M_wvp * Q * M_wvp.T
C_star = (-1/C_star[2,2])*C_star

ellipse_centre = Matrix([-C_star[0,2],-C_star[1,2]])

T_e_origin = eye(3)
T_e_origin[0:2, 2] = -ellipse_centre
C_centre = T_e_origin*C_star*T_e_origin.T
C_centre = (C_centre + C_centre.T)/2

C_submat = C_star[0:2, 0:2]

c1 = Symbol('c_00')
c2 = Symbol('c_01')
c3 = Symbol('c_10')
c4 = Symbol('c_11')
# Substitutions to reduce complexity
subs_c = {
    c1: C_submat[0,0],
    c2: C_submat[0,1],
    c3: C_submat[1,0],
    c4: C_submat[1,1],
}

temp = Matrix([[c1,c2], [c3,c4]])
lambdas = temp.eigenvals()
lambdas_list = []
for l in lambdas:
    lambdas_list.append(l)
semi_axes = Matrix(lambdas_list)
semi_axes = semi_axes.subs(subs_c)
# Simplify the formula for the axis, this is the maximum depth for semplification, 
# do that on other terms require too much time or do not have effect
semi_axes.simplify()
# [X_center, Y_center, semi_axis_A, semi_axis_B]
ellipse_state = Matrix([ellipse_centre[0], ellipse_centre[1], semi_axes[0], semi_axes[1]])


# Here we can obtain the final equations
ellipse_state = ellipse_state.subs(subs_wvp).subs(subs_ellipsoid)

In [23]:
# H MATRIX
jacobian = ellipse_state.jacobian(state)
H_matrix = jacobian.simplify()

In [ ]:
# With print is possible to print the code
# print(pycode(ax_a))
print("\\\\result is a eigen::Matrix<4,8,RowMajor>, delete last <<\nresult = <<")
for i in H_matrix:
    print("\t",ccode(i), "<<")
print(";")

In [ ]:
print(pycode(H_matrix))

In [17]:
for i in range(3):
    for j in range(4):
        pass
        #print("m_{{{}{}}} &= ".format(i,j), latex(M_wvp_real[i,j]), "\\\\")

In [18]:
# test values
subs_values = {
    x : 30,
    y : -20,
    theta : 0.5,
    l_ratio : 1,
    d_ratio : 1,
    h : 1,
    vr00 : 1,
    vr01 : 0.7,
    vr02 : 0.24,
    vr10 : 2.4,
    vr11 : 1,
    vr12 : 1.2,
    vr20 : 0.7,
    vr21 : 0.5,
    vr22 : 1,
    vtx : 1,
    vty : 1,
    vtz : 1,
    fx : 295.6,
    fy : 295.6,
    cx : 512,
    cy : 256
}

In [35]:
import math
# Define a function from pycode()
def compute_jacobian(x,y,theta,l_ratio,d_ratio,h,vr00,vr01,vr02,vr10,vr11,vr12,vr20,vr21,vr22,vtx,vty,vtz,fx,fy,cx,cy):
    return ImmutableDenseMatrix([[2*vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (-vr20*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) - (cx*vr20 + fx*vr00)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 2*vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (-vr21*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) - (cx*vr21 + fx*vr01)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + ((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.cos(theta) - (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*d_ratio**2*h**2*(-vr20*math.cos(theta) - vr21*math.sin(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/2*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/2*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (-1/2*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h*vr22**2 + vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*vr22*(cx*vr22 + fx*vr02) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + ((1/2)*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta)) + (1/2)*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) + (1/2)*h*vr22*(cx*vr22 + fx*vr02) - 1/2*vr22*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) - ((1/2)*cx*vr22 + (1/2)*fx*vr02)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 0, 0], [2*vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (-vr20*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - (cy*vr20 + fy*vr10)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 2*vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (-vr21*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - (cy*vr21 + fy*vr11)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + ((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.cos(theta) - (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*d_ratio**2*h**2*(-vr20*math.cos(theta) - vr21*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/2*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/2*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (-1/2*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h*vr22**2 + vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*vr22*(cy*vr22 + fy*vr12) - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + ((1/2)*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/2)*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*h*vr22*(cy*vr22 + fy*vr12) - 1/2*vr22*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - ((1/2)*cy*vr22 + (1/2)*fy*vr12)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 0, 0], [vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(2*vr20*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(2*vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*(2*cx*vr20 + 2*fx*vr00)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cx*vr20 + 2*fx*vr00)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cy*vr20 + 2*fy*vr10)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(2*cy*vr20 + 2*fy*vr10)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) + 4*(-2*(cx*vr20 + fx*vr00)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(cy*vr20 + fy*vr10)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + (2*cx*vr20 + 2*fx*vr00)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (2*cy*vr20 + 2*fy*vr10)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(2*vr21*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(2*vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*(2*cx*vr21 + 2*fx*vr01)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cx*vr21 + 2*fx*vr01)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cy*vr21 + 2*fy*vr11)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(2*cy*vr21 + 2*fy*vr11)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) + 4*(-2*(cx*vr21 + fx*vr01)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(cy*vr21 + fy*vr11)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + (2*cx*vr21 + 2*fx*vr01)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (2*cy*vr21 + 2*fy*vr11)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (1/2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*(-1/2*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/2*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - 2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + ((1/2)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.cos(theta) - (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.cos(theta) - (cx*vr21 + fx*vr01)*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/2)*h**2*l_ratio**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/4*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/2*h**2*l_ratio*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/2*h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/2*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))) - h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/4*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/2*d_ratio*h**2*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/2*d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/2*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))) - d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (1/2)*(-1/2*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h*vr22**2 + vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/2*d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/2*d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/2*h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/2*h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/2*h*(cx*vr22 + fx*vr02)**2 - 1/2*h*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*(-d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - h*vr22**2 + 2*vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*((1/2)*d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/2)*h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/2)*h*(cx*vr22 + fx*vr02)**2 - (cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + (d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + h*(cx*vr22 + fx*vr02)**2 - 2*(cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - 2*((1/2)*d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/2)*h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/2)*h*(cy*vr22 + fy*vr12)**2 - (cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + (d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + h*(cy*vr22 + fy*vr12)**2 - 2*(cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*(d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + h*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - 2*((1/2)*cx*vr22 + (1/2)*fx*vr02)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*((1/2)*cy*vr22 + (1/2)*fy*vr12)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/4*h**2*vr22**2 + ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/2*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h*vr22**2 + vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)) + (cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 0, 0], [vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-2*vr20*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(2*vr20*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*(2*cx*vr20 + 2*fx*vr00)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cx*vr20 + 2*fx*vr00)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cy*vr20 + 2*fy*vr10)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(2*cy*vr20 + 2*fy*vr10)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) + 4*(-2*(cx*vr20 + fx*vr00)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(cy*vr20 + fy*vr10)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + (2*cx*vr20 + 2*fx*vr00)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (2*cy*vr20 + 2*fy*vr10)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-2*vr21*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*h*vr22 + vr20*x + vr21*y + vtz) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(2*vr21*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*(2*cx*vr21 + 2*fx*vr01)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cx*vr21 + 2*fx*vr01)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + 2*(2*cy*vr21 + 2*fy*vr11)*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(2*cy*vr21 + 2*fy*vr11)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) + 4*(-2*(cx*vr21 + fx*vr01)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*(cy*vr21 + fy*vr11)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + (2*cx*vr21 + 2*fx*vr01)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (2*cy*vr21 + 2*fy*vr11)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (1/2)*(-1/4*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)) - 1/4*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*(-1/2*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) - 1/2*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta)))*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-2*(cx*vr20 + fx*vr00)*math.cos(theta) - 2*(cx*vr21 + fx*vr01)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*(-2*(cx*vr20 + fx*vr00)*math.sin(theta) + 2*(cx*vr21 + fx*vr01)*math.cos(theta))*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - 2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + ((1/2)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*(-2*(cy*vr20 + fy*vr10)*math.cos(theta) - 2*(cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*(-2*(cy*vr20 + fy*vr10)*math.sin(theta) + 2*(cy*vr21 + fy*vr11)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.cos(theta) - (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.cos(theta) - (cx*vr21 + fx*vr01)*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/2)*h**2*l_ratio**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/2)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))*(-2*vr20*math.cos(theta) - 2*vr21*math.sin(theta)) + (1/4)*h**2*l_ratio**2*(-2*vr20*math.sin(theta) + 2*vr21*math.cos(theta))*(vr20*math.cos(theta) + vr21*math.sin(theta))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/4*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*((1/2)*h**2*l_ratio*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/2*h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/2*h**2*l_ratio*(vr20*math.cos(theta) + vr21*math.sin(theta))**2*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*h**2*l_ratio*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))) - h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + h**2*l_ratio*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), -1/4*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*((1/2)*d_ratio*h**2*math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/2*d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*(-1/2*d_ratio*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*d_ratio*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))) - d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + d_ratio*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), (1/2)*(-1/2*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - 1/2*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - 1/2*h*vr22**2 + vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*(-1/4*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/4*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/4*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/4*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/4*h**2*(cx*vr22 + fx*vr02)**2 - 1/4*h**2*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2) + (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2 + (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2 + (1/2)*(-1/2*d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 - 1/2*d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 - 1/2*h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 - 1/2*h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 - 1/2*h*(cx*vr22 + fx*vr02)**2 - 1/2*h*(cy*vr22 + fy*vr12)**2 + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*(-d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 - h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 - h*vr22**2 + 2*vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz))*(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3 + (1/2)*(-2*((1/2)*d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/2)*h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/2)*h*(cx*vr22 + fx*vr02)**2 - (cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + (d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + h*(cx*vr22 + fx*vr02)**2 - 2*(cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) - 2*((1/2)*d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/2)*h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/2)*h*(cy*vr22 + fy*vr12)**2 - (cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2) + (d_ratio**2*h*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + h*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + h*(cy*vr22 + fy*vr12)**2 - 2*(cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))*(d_ratio**2*h*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + h*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + h*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - 2*((1/2)*cx*vr22 + (1/2)*fx*vr02)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)) - 2*((1/2)*cy*vr22 + (1/2)*fy*vr12)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**3/(((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2) + math.sqrt((((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)**2 - 2*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))**2 + (1/4)*h**2*(cx*vr22 + fx*vr02)**2 - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))**2)*((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2) + ((1/4)*d_ratio**2*h**2*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta))**2 + (1/4)*h**2*(cy*vr22 + fy*vr12)**2 - (cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11))**2)**2 + 4*((1/4)*d_ratio**2*h**2*(-(cx*vr20 + fx*vr00)*math.sin(theta) + (cx*vr21 + fx*vr01)*math.cos(theta))*(-(cy*vr20 + fy*vr10)*math.sin(theta) + (cy*vr21 + fy*vr11)*math.cos(theta)) + (1/4)*h**2*l_ratio**2*((cx*vr20 + fx*vr00)*math.cos(theta) + (cx*vr21 + fx*vr01)*math.sin(theta))*((cy*vr20 + fy*vr10)*math.cos(theta) + (cy*vr21 + fy*vr11)*math.sin(theta)) + (1/4)*h**2*(cx*vr22 + fx*vr02)*(cy*vr22 + fy*vr12) - (cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01))*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))**2)/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2)**2)*((1/2)*d_ratio**2*h*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/2)*h*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/2)*h*vr22**2 - vr22*((1/2)*h*vr22 + vr20*x + vr21*y + vtz)) + (cx*vr22 + fx*vr02)*(cx*vtz + fx*vtx + (1/2)*h*(cx*vr22 + fx*vr02) + x*(cx*vr20 + fx*vr00) + y*(cx*vr21 + fx*vr01)) + (cy*vr22 + fy*vr12)*(cy*vtz + fy*vty + (1/2)*h*(cy*vr22 + fy*vr12) + x*(cy*vr20 + fy*vr10) + y*(cy*vr21 + fy*vr11)))/((1/4)*d_ratio**2*h**2*(-vr20*math.sin(theta) + vr21*math.cos(theta))**2 + (1/4)*h**2*l_ratio**2*(vr20*math.cos(theta) + vr21*math.sin(theta))**2 + (1/4)*h**2*vr22**2 - ((1/2)*h*vr22 + vr20*x + vr21*y + vtz)**2), 0, 0]])


In [ ]:
# Test function with values
for i in range(10):
    print(compute_jacobian(x = i,
        y = -20,
        theta = 0.5,
        l_ratio = 1,
        d_ratio = 1,
        h = 1,
        vr00 = 1,
        vr01 = 0.7,
        vr02 = 0.24,
        vr10 = 2.4,
        vr11 = 1,
        vr12 = 1.2,
        vr20 = 0.7,
        vr21 = 0.5,
        vr22 = 1,
        vtx = 1,
        vty = 1,
        vtz = 1,
        fx = 295.6,
        fy = 295.6,
        cx = 512,
        cy = 256))
